# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [9]:
import os
from glob import glob

PRICE_DATA = os.getenv('PRICE_DATA')

parquet_files = glob(os.path.join(PRICE_DATA, '**', '*.parquet'), recursive=True)


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [18]:
dd_px = dd.read_parquet(parquet_files).set_index("Ticker")
dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1)).assign(
            Adj_Close_lag_1 = x['Adj Close'].shift(1))
)
dd_feat = dd_shift.assign(
    returns = lambda x: x['Close'] / x['Close_lag_1'] - 1
).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
)

/tmp/ipykernel_2573/870707631.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [22]:
import pandas as pd
df_feat = dd_feat.compute()
df_feat['returns_ma'] = df_feat['returns'].rolling(10).mean()
print(df_feat)


Price                       Date  Adj Close      Close       High        Low  \
Ticker                                                                         
DOV    2003-01-02 00:00:00+00:00  13.402376  20.418425  20.773701  19.567097   
DOV    2003-01-03 00:00:00+00:00  13.362778  20.358093  20.472050  20.170399   
DOV    2003-01-06 00:00:00+00:00  13.732377  20.921175  21.068649  20.344687   
DOV    2003-01-07 00:00:00+00:00  13.587181  20.699965  20.947989  20.559195   
DOV    2003-01-08 00:00:00+00:00  13.265972  20.210619  20.699965  20.177103   
...                          ...        ...        ...        ...        ...   
CTLT   2004-12-27 00:00:00+00:00        NaN        NaN        NaN        NaN   
CTLT   2004-12-28 00:00:00+00:00        NaN        NaN        NaN        NaN   
CTLT   2004-12-29 00:00:00+00:00        NaN        NaN        NaN        NaN   
CTLT   2004-12-30 00:00:00+00:00        NaN        NaN        NaN        NaN   
CTLT   2004-12-31 00:00:00+00:00        

In [28]:

print(df_feat.head(500))

Price                       Date  Adj Close      Close       High        Low  \
Ticker                                                                         
DOV    2003-01-02 00:00:00+00:00  13.402376  20.418425  20.773701  19.567097   
DOV    2003-01-03 00:00:00+00:00  13.362778  20.358093  20.472050  20.170399   
DOV    2003-01-06 00:00:00+00:00  13.732377  20.921175  21.068649  20.344687   
DOV    2003-01-07 00:00:00+00:00  13.587181  20.699965  20.947989  20.559195   
DOV    2003-01-08 00:00:00+00:00  13.265972  20.210619  20.699965  20.177103   
...                          ...        ...        ...        ...        ...   
DOV    2001-12-24 00:00:00+00:00  16.015343  24.795715  24.876156  24.701868   
DOV    2001-12-26 00:00:00+00:00  15.985035  24.748793  24.869452  24.668350   
DOV    2001-12-27 00:00:00+00:00  16.019672  24.802420  24.802420  24.688461   
DOV    2001-12-28 00:00:00+00:00  16.036995  24.829231  24.949892  24.641539   
DOV    2001-12-31 00:00:00+00:00  16.049

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

We could set up the moving average calculation using dask. It would be better to do this in dask since it can perform the computation in a distributed manner without needing to bring the entire dataset into memory at once.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.